This is script will figure out travel times from different runs

In [2]:
import argparse
import pandas

# declare some column names
travelTime = 'travelTime'
startTime = 'startTime'
endTime = 'endTime'
id = 'personId'
tripNumber = 'tripNo'
mode = 'mainMode'

def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--files', nargs='+', required=True)
    parser.add_argument('-n', '--names', nargs='+', required=True)
    return parser.parse_args(args)

def read_csv(listOfFiles, listOfNames):
    
    assert(len(listOfFiles) == len(listOfNames)), 'number of files and scenario names should be equal but was: (' + str(len(listOfFiles)) + '/' + str(len(listOfNames)) + ')'
    
    result_data_frame = pandas.DataFrame({id: [], tripNumber: []})

    for file, name in zip(listOfFiles, listOfNames):
        scenario = pandas.read_csv(file, usecols=[id, tripNumber, startTime, endTime, mode])
        scenario[get_column_name(travelTime, name)] = scenario[endTime] - scenario[startTime]
        scenario.rename(
            columns={startTime: get_column_name(startTime, name), endTime: get_column_name(endTime, name), mode:get_column_name(mode, name)}, 
            inplace=True)
        result_data_frame = pandas.merge(scenario, result_data_frame, on=[id, tripNumber], how='outer')
    return result_data_frame

def copy_travel_times(dataFrame, names):
    
    result = pandas.DataFrame()
    for name in names:
        result[get_column_name(travelTime, name)] = dataFrame[get_column_name(travelTime, name)]
        result[get_column_name(mode, name)] = dataFrame[get_column_name(mode, name)]
    return result
        
def get_column_name(column, scenario):
    
    return column + '_' + scenario


In [80]:
dataFolder = "C:\\Users\\Janek\\Desktop\\nemo_analysis\\"
args = parse_args([
    '-f', dataFolder + 'baseCase_continued.csv', dataFolder + 'bike-highways.csv',
    '-n', 'Base Case', 'Bike Highways' 
])

data = read_csv(args.files, args.names)
a = data[data[id] == 394783401]
print(a)

print(data)

    personId  tripNo  startTime_Bike Highways  endTime_Bike Highways  \
0  394783401       0                  39594.0                40353.0   
1  394783401       1                  40843.0                41602.0   

  mainMode_Bike Highways  travelTime_Bike Highways  startTime_Base Case  \
0                   walk                     759.0              39594.0   
1                   walk                     759.0              40843.0   

   endTime_Base Case mainMode_Base Case  travelTime_Base Case  
0            40353.0               walk                 759.0  
1            41602.0               walk                 759.0  
          personId  tripNo  startTime_Bike Highways  endTime_Bike Highways  \
0        394783401       0                  39594.0                40353.0   
1        394783401       1                  40843.0                41602.0   
2       1414039501       0                  24600.0                26803.0   
3       1414039501       1                  36300.0  

In [3]:
# get the average travel time by mode
travelTimes = copy_travel_times(data, args.names)
grouped = travelTimes.groupby(get_column_name(mode, args.names[0]))
print(grouped)





NameError: name 'data' is not defined